# Google Colab

Jupyter notebook setting up Google Colab to work as a remote Jupyter server and
installs some QoL goodies.

## Use Google Colab as a Jupyter Notebook Server

In [ ]:
async def expose_jupyter_server(expose=False):
    """
    Expose Jupyter server running on Google Colab via ngrok. Not executed by
    default to avoid exposing the server unintentionally. To expose the server,
    set `expose` to `True` and call the function.
    """
    if not expose:
        print("Not exposing Jupyter server.")
        return
    
    try:
        from google.colab import userdata
        import re
    except ImportError:
        print("Not running on Google Colab! Not exposing Jupyter server.")
        return

    # Install jupyterlab and ngrok
    !pip install jupyterlab==2.2.9 ngrok -q

    # Run jupyterlab in background
    !nohup jupyter lab --no-browser --port=8888 --ip=0.0.0.0 &

    with open("nohup.out", "r") as file:
        match = None
        while not match:
            content = file.read()
            match = re.findall("\?token=.*", content)
        token = match[-1]

    # Make jupyterlab accessible via ngrok
    import ngrok

    listener = await ngrok.forward(
        8888, 
        # domain=userdata.get('NGROKDOMAIN'), 
        authtoken=userdata.get("NGROKTOKEN")
    )
    print("Connect to URL:", listener.url() + token)

await expose_jupyter_server(expose=True)

## Automatically Use and Install Conda if in Google Colab

In [ ]:
import sys
import shutil

def has_conda():
    return shutil.which("conda") is not None

def install_conda():
    !pip install -q condacolab
    import condacolab
    condacolab.install()

if not has_conda():
    if "google.colab" in sys.modules:
        install_conda()
    else:
        raise RuntimeError("""
            Conda not found, and cannot be automatically installed unless
            in a Google Colab environment. Please install conda or launch
            in Google Colab.
        """)

## Prevent Google Colab from Disconnecting

To prevent google colab from quitting when an experiment is running, you can add
enter the following code in the _developer console_ (Ctrl+Shift+I in Google Chrome):

```javascript
var startClickConnect = function startClickConnect(){
    var clickConnect = function clickConnect(){
        console.log("Connnect Clicked - Start");
        document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
        console.log("Connnect Clicked - End"); 
    };

    var intervalId = setInterval(clickConnect, 60000);

    var stopClickConnectHandler = function stopClickConnect() {
        console.log("Connnect Clicked Stopped - Start");
        clearInterval(intervalId);
        console.log("Connnect Clicked Stopped - End");
    };

    return stopClickConnectHandler;
};

var stopClickConnect = startClickConnect();
```

In order to stop, call:

```javascript
stopClickConnect();
```

Source: https://stackoverflow.com/questions/57113226/how-can-i-prevent-google-colab-from-disconnecting